## Prepare the inputs

- encoder_input_data: 2D array of shape `(num_images * 5, 512)`
- decoder_input_data: 3D array of shape `(num_captions, max_words_in_sentence, num_words)`
- decoder_output_data: same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]

## Image Preprocessing

obtain bottleneck features

In [1]:
import numpy as np

In [2]:
# Since there are 5 captions per image, duplicate the bottleneck features
def duplicate_bottleneck_features(features):
    num_captions = 5 # 5 stands for number of captions per image
    num_rows = features.shape[0] * num_captions 

    features_dup = np.zeros((num_rows, features.shape[1]))
    for i, image in enumerate(features):
        for j in range(num_captions):
            features_dup[i*num_captions + j] = image
    return features_dup    

In [3]:
bottleneck_features = np.load('bottleneck_features/Flicker8k_bottleneck_features_VGG16_avgpooling.npz')
bottleneck_features_train = bottleneck_features["train"]
bottleneck_features_validation = bottleneck_features["validation"]
bottleneck_features_test = bottleneck_features["test"]

bottleneck_features_train_dup = duplicate_bottleneck_features(bottleneck_features_train)
bottleneck_features_validation_dup = duplicate_bottleneck_features(bottleneck_features_validation)
bottleneck_features_test_dup = duplicate_bottleneck_features(bottleneck_features_test)

In [4]:
print(bottleneck_features_train_dup.shape)
print(bottleneck_features_validation_dup.shape)
print(bottleneck_features_test_dup.shape)

(30000, 512)
(5000, 512)
(5000, 512)


## Word Embedding

In [5]:
from caption_utils import *
train_fns_list, dev_fns_list, test_fns_list = load_split_lists()

train_captions_raw, dev_captions_raw, test_captions_raw = get_caption_split()
vocab = create_vocab(train_captions_raw)
token2idx, idx2token = vocab_to_index(vocab)     
captions_data = (train_captions_raw.copy(), dev_captions_raw.copy(), test_captions_raw.copy())
train_captions, dev_captions, test_captions = process_captions(captions_data, token2idx)

/home/ubuntu/anaconda3/envs/tensorflow-3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
print(len(vocab))

2531


## Caption Preprocessing


In [7]:
captions_onehot = np.load('preprocessed_captions/Flicker8k_onehot_'+str(len(vocab))+'_words.npz')
train_captions_onehot = captions_onehot["train"]
validation_captions_onehot = captions_onehot["validation"]
test_captions_onehot = captions_onehot["test"]

train_captions_onehot = train_captions_onehot.astype(np.float32)
validation_captions_onehot = validation_captions_onehot.astype(np.float32)
test_captions_onehot = test_captions_onehot.astype(np.float32)

In [8]:
print(train_captions_onehot.shape)
print(validation_captions_onehot.shape)
print(test_captions_onehot.shape)

(30000, 39, 2531)
(5000, 39, 2531)
(5000, 39, 2531)


In [9]:
print(train_captions_onehot.dtype, validation_captions_onehot.dtype, test_captions_onehot.dtype)

float32 float32 float32


In [10]:
def captions_onehot_split(captions_onehot):
    """ returns decoder input data and decoder target data """
    return captions_onehot[:, :-1, :], captions_onehot[:, :, :]

### Training captions - > decoder input, target data  

In [11]:
train_decoder_input, train_decoder_target = captions_onehot_split(train_captions_onehot)
validation_decoder_input, validation_decoder_target = captions_onehot_split(validation_captions_onehot)
test_decoder_input, test_decoder_target = captions_onehot_split(test_captions_onehot)

In [12]:
train_encoder_output = bottleneck_features_train_dup.astype(np.float32)
test_encoder_output = bottleneck_features_test_dup.astype(np.float32)
validation_encoder_output = bottleneck_features_validation_dup.astype(np.float32)

In [13]:
test_decoder_input = np.argmax(test_decoder_input, axis=-1)
train_decoder_input = np.argmax(train_decoder_input, axis=-1)
validation_decoder_input = np.argmax(validation_decoder_input, axis=-1)

In [14]:
print("Decoder Input", train_decoder_input.shape, train_decoder_input.dtype)
print("Decoder Target", train_decoder_target.shape, train_decoder_target.dtype)
print("Encoder Output", train_encoder_output.shape, train_encoder_output.dtype)

Decoder Input (30000, 38) int64
Decoder Target (30000, 39, 2531) float32
Encoder Output (30000, 512) float32


## Build Model

-------------------------
# Experiment 

## Parameters

In [15]:
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, GRU, Dense, Embedding, BatchNormalization, RepeatVector, Concatenate, TimeDistributed, Merge, Dropout
from keras import backend as K

In [16]:
emb_size = 150
lstm_size = 300
vocab_size = len(vocab)
max_length = train_decoder_target.shape[1]
print(max_length)

39


In [17]:
K.clear_session()

# Image -> Image embedding
img_emb = Sequential()
img_emb.add(Dense(emb_size, input_dim=train_encoder_output.shape[1], activation='relu'))
img_emb.add(RepeatVector(1))
img_emb.add(Dropout(0.2))

# Sentence to Word embedding
word_emb = Sequential()
word_emb.add(Embedding(input_dim=vocab_size, output_dim=emb_size, input_length=max_length-1))
word_emb.add(Dropout(0.2))

# Merge img_emb and word_emb
seq_in = Sequential()
seq_in.add(Merge([img_emb, word_emb], mode='concat', concat_axis=1))
seq_in.add(Dropout(0.2))
#seq_in.add(Concatenate([img_emb, word_emb]))

# RNN Layer
seq_in.add(GRU(lstm_size, return_sequences=True))
seq_in.add(Dropout(0.2))
seq_in.add(TimeDistributed(Dense(vocab_size)))

seq_in.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

/home/ubuntu/anaconda3/envs/tensorflow-3.5/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()


In [18]:
print(img_emb.summary())
print(word_emb.summary())
print(seq_in.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 150)               76950     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 1, 150)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 150)            0         
Total params: 76,950
Trainable params: 76,950
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 38, 150)           379650    
_________________________________________________________________
dropout_2 (Dropout)          (None, 38, 150)           0         
Total params: 379,650
Trainable params: 379,650
Non-trainable param

In [19]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.hdf5', 
                               verbose=1, save_best_only=True)

In [20]:
seq_in.fit([train_encoder_output, train_decoder_input], [train_decoder_target], 
           validation_data=([validation_encoder_output, validation_decoder_input], [validation_decoder_target]),
           epochs=50, batch_size=32, callbacks=[checkpointer], verbose=1)

Train on 30000 samples, validate on 5000 samples
Epoch 1/50
30000/30000 [==============================] - 65s 2ms/step - loss: 3.6376 - acc: 0.0276 - val_loss: 3.3095 - val_acc: 0.0285

Epoch 00001: val_loss improved from inf to 3.30947, saving model to saved_models/weights.best.VGG16.hdf5
Epoch 2/50
30000/30000 [==============================] - 63s 2ms/step - loss: 3.5916 - acc: 0.0287 - val_loss: 3.7013 - val_acc: 0.0286

Epoch 00002: val_loss did not improve
Epoch 3/50
30000/30000 [==============================] - 63s 2ms/step - loss: 3.5462 - acc: 0.0288 - val_loss: 3.7184 - val_acc: 0.0286

Epoch 00003: val_loss did not improve
Epoch 4/50
30000/30000 [==============================] - 63s 2ms/step - loss: 3.4928 - acc: 0.0295 - val_loss: 3.2444 - val_acc: 0.0302

Epoch 00004: val_loss improved from 3.30947 to 3.24435, saving model to saved_models/weights.best.VGG16.hdf5
Epoch 5/50
30000/30000 [==============================] - 63s 2ms/step - loss: 3.4725 - acc: 0.0304 - val_los

30000/30000 [==============================] - 62s 2ms/step - loss: 3.1378 - acc: 0.0410 - val_loss: 3.2404 - val_acc: 0.0416

Epoch 00044: val_loss did not improve
Epoch 45/50
30000/30000 [==============================] - 62s 2ms/step - loss: 3.1607 - acc: 0.0410 - val_loss: 3.0868 - val_acc: 0.0417

Epoch 00045: val_loss did not improve
Epoch 46/50
30000/30000 [==============================] - 63s 2ms/step - loss: 3.1524 - acc: 0.0411 - val_loss: 2.9293 - val_acc: 0.0419

Epoch 00046: val_loss did not improve
Epoch 47/50
30000/30000 [==============================] - 63s 2ms/step - loss: 3.1548 - acc: 0.0412 - val_loss: 3.4887 - val_acc: 0.0419

Epoch 00047: val_loss did not improve
Epoch 48/50
30000/30000 [==============================] - 63s 2ms/step - loss: 3.1583 - acc: 0.0412 - val_loss: 3.0571 - val_acc: 0.0418

Epoch 00048: val_loss did not improve
Epoch 49/50
30000/30000 [==============================] - 63s 2ms/step - loss: 3.1519 - acc: 0.0413 - val_loss: 3.2679 - val_a